In [1]:
from flask.json import jsonify
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [4]:
import requests


response = requests.get('http://192.168.0.17:5000/get-predict')
data = response.json()
print(data)

{'historical': [{'adjClose': 107.68, 'change': 0.35, 'changeOverTime': 0.0025598, 'changePercent': 0.25598, 'close': 137.08, 'date': '2019-04-08', 'high': 137.11, 'label': 'April 08, 19', 'low': 136.59, 'open': 136.73, 'unadjustedVolume': 2215637, 'volume': 2215637, 'vwap': 136.93}, {'adjClose': 106.72, 'change': -0.53, 'changeOverTime': -0.0038859, 'changePercent': -0.38859, 'close': 135.86, 'date': '2019-04-09', 'high': 136.66, 'label': 'April 09, 19', 'low': 135.6, 'open': 136.39, 'unadjustedVolume': 3178271, 'volume': 3178271, 'vwap': 136.04}, {'adjClose': 107.4, 'change': 0.78, 'changeOverTime': 0.0057374, 'changePercent': 0.57374, 'close': 136.73, 'date': '2019-04-10', 'high': 137.19, 'label': 'April 10, 19', 'low': 135.74, 'open': 135.95, 'unadjustedVolume': 2801188, 'volume': 2801188, 'vwap': 136.55}, {'adjClose': 107.97, 'change': -0.02, 'changeOverTime': -0.0001454757, 'changePercent': -0.01454757, 'close': 137.46, 'date': '2019-04-11', 'high': 137.77, 'label': 'April 11, 19'

In [5]:
def preprocess_data(data):
    try:
        # Convert JSON data to DataFrame
        df = pd.DataFrame(data['historical'])
        
        # Set 'date' column as index
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        
        # Select 1 feature  'close' column for prediction
        df = df[['close']]
        
        # Scale the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        data_scaled = scaler.fit_transform(df)
        
        return data_scaled, scaler
    except Exception as e:
        print("Error during data preprocessing:", e)
        return None, None

In [11]:
def define_model():
    # Define and compile the model architecture
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = define_model()


## preprocessing sections above ^
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)


def predict(ticker_data):
    try:
        processed_data, scaler = preprocess_data(ticker_data)

        time_step = 100
        X, y = create_dataset(processed_data, time_step)

        X = X.reshape((X.shape[0], X.shape[1], 1))
        
        prediction = model.predict(X)

        prediction_original_scale = inverse_transform_predictions(prediction, scaler)
        return prediction_original_scale
    except Exception as e:
        print("Error during prediction:", e)
        return None

def inverse_transform_predictions(predictions, scaler):
    predictions = np.array(predictions)
    if predictions.ndim == 1:
        predictions = predictions.reshape(-1, 1)
    predictions_original_scale = scaler.inverse_transform(predictions)
    return predictions_original_scale

## predict with correct output
def predict_with_date_and_column(ticker_data):
    try:
        processed_data, scaler = preprocess_data(ticker_data)

        time_step = 100
        X, y = create_dataset(processed_data, time_step)

        X = X.reshape((X.shape[0], X.shape[1], 1))
        
        prediction = model.predict(X)

        prediction_original_scale = inverse_transform_predictions(prediction, scaler)
        
        # Get the original dates from the ticker_data
        dates = [data['date'] for data in ticker_data['historical']]
        
        # Get the original close prices
        original_close_prices = [data['close'] for data in ticker_data['historical']]

        # DataFrame with dates and predictions
        df_predictions = pd.DataFrame({
            'date': dates[time_step:],  # Exclude initial rows used for prediction
            'original_close': original_close_prices[time_step:],  # Exclude initial rows used for prediction
            'predicted_close': prediction_original_scale.flatten()  # Flatten the predictions
        })
        
        # Rename the 'prediction' column with the original column name ('close')
        df_predictions.rename(columns={'prediction': 'close_prediction'}, inplace=True)
        
        return df_predictions
    except Exception as e:
        print("Error during prediction:", e)
        return None



# !works 
# prediction = predict(data)
# print("Predicted prices:", prediction)

prediction_with_format = predict_with_date_and_column(data)
print(prediction_with_format)

37/37 [==============================] - 2s 20ms/step
            date  original_close  predicted_close
0     2019-08-29          128.95        96.207230
1     2019-08-30          129.57        96.131516
2     2019-09-03          128.20        96.074600
3     2019-09-04          130.33        96.032768
4     2019-09-05          134.77        96.007347
...          ...             ...              ...
1153  2024-04-01          189.83       105.567581
1154  2024-04-02          188.88       105.545311
1155  2024-04-03          190.90       105.518715
1156  2024-04-04          187.94       105.492340
1157  2024-04-05          189.14       105.462769

[1158 rows x 3 columns]
